In [1]:
import sys
sys.path.append('..')

import math
import os
import json
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import config

from nab.runner import Runner
from nab.util import (detectorNameToClass, checkInputs)

In [5]:
detectors=["null"]

def getDetectorClassConstructors(detectors):
  """
  Takes in names of detectors. Collects class names that correspond to those
  detectors and returns them in a dict. The dict maps detector name to class
  names. Assumes the detectors have been imported.
  """
  detectorConstructors = {
  d : globals()[detectorNameToClass(d)] for d in detectors}

  return detectorConstructors

if "bayesChangePt" in detectors:
    from nab.detectors.bayes_changept.bayes_changept_detector import (
      BayesChangePtDetector)
if "null" in detectors:
    from nab.detectors.null.null_detector import NullDetector
if "random" in detectors:
    from nab.detectors.random.random_detector import RandomDetector
  # By default the skyline detector is disabled, it can still be added to the
  # detectors argument to enable it, for more info see #335 and #333
if "skyline" in detectors:
    from nab.detectors.skyline.skyline_detector import SkylineDetector
if "windowedGaussian" in detectors:
    from nab.detectors.gaussian.windowedGaussian_detector import (
      WindowedGaussianDetector)
if "knncad" in detectors:
    from nab.detectors.knncad.knncad_detector import KnncadDetector
if "relativeEntropy" in detectors:
    from nab.detectors.relative_entropy.relative_entropy_detector import (
      RelativeEntropyDetector)
if "expose" in detectors:
    from nab.detectors.expose.expose_detector import ExposeDetector
if "contextOSE" in detectors:
    from nab.detectors.context_ose.context_ose_detector import (
    ContextOSEDetector )
if "earthgeckoSkyline" in detectors:
    from nab.detectors.earthgecko_skyline.earthgecko_skyline_detector import EarthgeckoSkylineDetector
if "ARTime" in detectors:
    from nab.detectors.ARTime.ARTime_detector import ARTimeDetector

In [6]:
root = config.NAB_ASSETS
#don't have .. in path as that messup up relative paths in nab code

dataDir= str(root / "raw_datasets")
windowsFile=str(root / "labels" / "combined_windows.json")
resultsDir=str(root / "predictions" )
profilesFile=str(root / "config" / "profiles.json")
thresholdsFile=str(root / "config" / "config.json")
numCPUs=4

runner = Runner(dataDir=dataDir,
                labelPath=windowsFile,
                resultsDir=resultsDir,
                profilesPath=profilesFile,
                thresholdPath=thresholdsFile,
                numCPUs=numCPUs)

runner.initialize()

In [ ]:
detectorConstructors = getDetectorClassConstructors(detectors)
runner.detect(detectorConstructors)


Running detection step


In [6]:
runner.optimize(detectors)


Running optimize step
Optimizer found a max score of 14.19823623760221 with anomaly threshold 1.0.
Optimizer found a max score of -20.058228224940617 with anomaly threshold 1.0.
Optimizer found a max score of -10.441429686808725 with anomaly threshold 0.9972640218878248.


{'knncad': {'standard': {'threshold': 1.0, 'score': 14.19823623760221},
  'reward_low_FP_rate': {'threshold': 1.0, 'score': -20.058228224940617},
  'reward_low_FN_rate': {'threshold': 0.9972640218878248,
   'score': -10.441429686808725}}}

In [7]:
with open(thresholdsFile) as thresholdConfigFile:
    detectorThresholds = json.load(thresholdConfigFile)
runner.score(detectors, detectorThresholds)


Running scoring step
knncad detector benchmark scores written to static\nab\results\knncad\knncad_standard_scores.csv
knncad detector benchmark scores written to static\nab\results\knncad\knncad_reward_low_FP_rate_scores.csv
knncad detector benchmark scores written to static\nab\results\knncad\knncad_reward_low_FN_rate_scores.csv
